In [17]:
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.decomposition import PCA
plt.rcParams['font.sans-serif']=['SimHei'] #图中文字体设置为黑体
plt.rcParams['axes.unicode_minus']=False
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score
import scipy
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import math
from scipy.stats import entropy

In [18]:
def read_data(file_name):
    df = pd.read_csv(file_name)
    df = df.drop(labels=['BASIC_entity_name', 'BASIC_year'], axis=1)
    fields_df = pd.read_excel('IDOU_05_事件触发_03_大小类事件映射表_v2.0.xlsx')
    rename_dict = {}
    for i, row in fields_df.iterrows():
        rename_dict[row['大类事件代码'] + '_' + row['小类事件代码']] = row['名称']
    df = df.rename(columns=rename_dict)
    if 'REVENUEAB_test' in df.columns:
        df = df.drop(labels='REVENUEAB_test', axis=1)
    return df

In [19]:
df_all = read_data('舞弊案例全市场数据2014-2021.csv')
df_part_1 = read_data('all_samples.csv')
df_part_2 = read_data('舞弊案例数据库.csv')

In [20]:
df = df_all

In [21]:
result = pd.DataFrame(columns=df.columns, index=df.columns)
for col in df.columns:
    for other_col in df.columns:
        if other_col != col:
            pca = PCA(n_components=1)
            pca_df = pca.fit_transform(df[[col, other_col]])
            corr = pca.explained_variance_ratio_[0]
            result.loc[col, other_col] = corr
            if corr > 0.8:
                print(col, other_col, corr)

资产质量异常-商誉质量存疑 筹资活动异常-存贷双高 0.8354684683565012
资产质量异常-商誉质量存疑 关联投资异常-商誉质量 0.8066192069101751
资产质量异常-商誉质量存疑 成本费用异常-可操纵应计费用调节 0.81248610001443
资产质量异常-商誉质量存疑 成本费用异常-会计政策调节 0.8243416447005603
资产质量异常-商誉质量存疑 关联投资异常-子公司资金支持 0.828198326702928
资产质量异常-商誉质量存疑 资产质量异常-大额长期应收款 0.9377188988449592
盈余质量异常-收入结构异常 成本费用异常-可操纵应计费用调节 0.8518428840664933
盈余质量异常-收入结构异常 成本费用异常-会计政策调节 0.8607225517214171
盈余质量异常-收入结构异常 关联投资异常-子公司资金支持 0.8632060618325205
盈余质量异常-收入结构异常 资产质量异常-大额长期应收款 0.9185606164071172
成本费用异常-期间费用调节 筹资活动异常-存贷双高 0.903342832568852
成本费用异常-期间费用调节 财务困境-经营活动现金净流量 0.8308074252294357
成本费用异常-期间费用调节 资产质量异常-大额长期应收款 0.9647773793133817
筹资活动异常-存贷双高 资产质量异常-商誉质量存疑 0.8354684683565164
筹资活动异常-存贷双高 成本费用异常-期间费用调节 0.9033428325688183
筹资活动异常-存贷双高 收入增长异常-赊销比例提高 0.9182567969474348
筹资活动异常-存贷双高 关联方交易异常-关联方占款风险 0.8736498278429153
筹资活动异常-存贷双高 收入增长异常-赊销质量 0.8982579184493283
筹资活动异常-存贷双高 现金质量异常-货币资金受限 0.8793404344550403
筹资活动异常-存贷双高 投资活动异常-建设投资扩张 0.9169682442661925
筹资活动异常-存贷双高 关联投资异常-商誉质量 0.8761937695510014
筹资活动异常-存贷双高 成本费用异常-采购现金流不匹配 0

In [22]:
result.to_excel('PCA结果.xlsx')